<a href="https://colab.research.google.com/github/jaeryong77/NLP_sentimental/blob/main/%5BBASE_LINE%5D_%EC%98%81%EB%AC%B8%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D%EA%B3%BC%EC%A0%9C_2%EC%B0%A8_%EC%A0%9C%EC%B6%9C%EB%B2%84%EC%A0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Setting 
!pip install transformers --quiet # package installer for python

# Data Set 을 가져온다 
import json

data = {'train': {'speaker': [], 'utterance': [], 'emotion': []},
        'dev': {'speaker': [], 'utterance': [], 'emotion': []},
        'test': {'speaker': [], 'utterance': [], 'emotion': []}}

for dtype in ['train', 'dev', 'test']:
  for dialog in json.loads(open('/content/gdrive/My Drive/Colab Notebooks/friend_eng/friends_'+dtype+'.json').read()):
    for line in dialog:
      data[dtype]['speaker'].append(line['speaker'])
      data[dtype]['utterance'].append(line['utterance'])
      data[dtype]['emotion'].append(line['emotion'])
print('data load success')

e2i_dict = dict((emo, i) for i, emo in enumerate(set(data['train']['emotion'])))
i2e_dict = {i: e for e, i in e2i_dict.items()}
print('e2i_dic success')

In [ ]:

import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
model = BertModel.from_pretrained(pretrained_weights)

class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert_tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
    self.bert_model = BertModel.from_pretrained(pretrained_weights)
    self.linear = torch.nn.Linear(768, len(e2i_dict))

  def forward(self, utterance):
    tokens = self.bert_tokenizer.tokenize(utterance)
    tokens = ['[CLS]'] + tokens + ['[SEP]'] # (len)
    ids = [tokenizer.convert_tokens_to_ids(tokens)] # (bat=1, len)
    input_tensor = torch.tensor(ids).cuda()

    hidden_tensor = self.bert_model(input_tensor)[0] # (bat, len, hid)
    hidden_tensor = hidden_tensor[:, 0, :] # (bat, hid)
    logit = self.linear(hidden_tensor)
    return logit

from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(true_list, pred_list):
  precision = precision_score(true_list, pred_list, average=None)
  recall = recall_score(true_list, pred_list, average=None)
  micro_f1 = f1_score(true_list, pred_list, average='micro')
  print('precision:\t', ['%.4f' % v for v in precision])
  print('recall:\t\t', ['%.4f' % v for v in recall])
  print('micro_f1: %.6f' % micro_f1)


pretrained_weights = 'bert-base-uncased'
learning_rate = 1e-5
n_epoch = 3

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
from tqdm import tqdm_notebook

model = Model()
model.cuda()
criterion = torch.nn.CrossEntropyLoss() # LogSoftmax & NLLLoss
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

for i_epoch in range(n_epoch):
  print('i_epoch:', i_epoch)

  model.train()
  for i_batch in tqdm_notebook(range(len(data['train']['utterance']))):
    logit = model(data['train']['utterance'][i_batch])
    target = torch.tensor([e2i_dict[data['train']['emotion'][i_batch]]]).cuda()
    loss = criterion(logit, target)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  
  model.eval()
  pred_list, true_list = [], []
  for i_batch in tqdm_notebook(range(len(data['dev']['utterance']))):
    logit = model(data['dev']['utterance'][i_batch])
    _, max_idx = torch.max(logit, dim=-1)
    pred_list += max_idx.tolist()
    true_list += [e2i_dict[data['dev']['emotion'][i_batch]]]
  print(i2e_dict)  
  evaluate(pred_list, true_list) # print results

In [ ]:
from collections import OrderedDict
import pandas as pd
en_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/competition_file/en_data.csv',encoding='utf-8')

result = []
labeled = []


# dialogs = json.loads(open('unlabeled.json').read())
for line in tqdm_notebook(en_data['utterance']):
  logit = model(line)
  _, max_idx = torch.max(logit, dim=-1)
  pred_emotion = max_idx.tolist()[0]
  result.append(i2e_dict[pred_emotion])

result_data=pd.DataFrame()
result_data['id'] = en_data['id']
result_data['Predicted'] = result

# File 생성
result_data.to_csv('/content/gdrive/My Drive/Colab Notebooks/competition_file/en_result.csv', encoding='utf-8', sep=',', index = False)

In [7]:
result_data=pd.DataFrame()
result_data['id'] = en_data['id']
result_data['Predicted'] = result

In [8]:
# File 생성
result_data.to_csv('/content/gdrive/My Drive/Colab Notebooks/competition_file/en_result.csv', encoding='utf-8', sep=',', index = False)